In [115]:
import boto3
from PIL import Image
from io import BytesIO
from resmem import ResMem, transformer
import time
import dataset
import pandas as pd
import numpy as np

In [116]:
def read_image_from_s3(bucket, key, region_name='us-east-1'):
    """Load image file from s3.

    Parameters
    ----------
    bucket: string
        Bucket name
    key : string
        Path in s3

    Returns
    -------
    np array
        Image array
    """
    s3 = boto3.resource('s3', region_name=region_name)
    bucket = s3.Bucket(bucket)
    object = bucket.Object(key)
    response = object.get()
    file_stream = response['Body']
    im = Image.open(file_stream)
    return im

In [117]:
rds = boto3.client('rds')

In [118]:
response = rds.describe_db_instances()
db = [ tmp_db for tmp_db in response['DBInstances'] if tmp_db['DBName'] == 'moviesdb'][0]
ENDPOINT = db['Endpoint']['Address']
PORT = db['Endpoint']['Port']
DBID = db['DBInstanceIdentifier']
username = 'yuetong'
password = 'password'
db_url = \
"mysql+mysqlconnector://{}:{}@{}:{}/moviesdb".format(username, password, ENDPOINT, PORT)

#db = dataset.connect(db_url)
db = None
while True:
    try:
        db = dataset.connect(db_url)
        break
    except:
        continue

In [11]:
db['movie_info'].

<Table(movie_info)>

In [14]:
import mysql.connector
conn =  mysql.connector.connect(host=ENDPOINT,
                                user=username,
                                passwd=password, 
                                port=PORT, 
                                database='moviesdb')
cur = conn.cursor()

imdb_info_dict = {'tconst': imdb_id, 'title': '',
                  'MPAA': '', 'genre': [], 'poster_url': '',
                  'imdb_rating': '', 'num_imdb_votes': '',

In [113]:
cur.execute('''SELECT * from movie_info''')
query_results = cur.fetchall()

df_all = pd.DataFrame(query_results)
df_all.head()

,0,1,2,3,4,5,6,7,8
0,1,tt0011237,"Der Golem, wie er in die Welt kam",Unrated,"Fantasy,Horror",https://m.media-amazon.com/images/M/MV5BMTQ5MT...,7.2,8107.0,0.748338
1,2,tt0011237,"Der Golem, wie er in die Welt kam",Unrated,"Fantasy,Horror",https://m.media-amazon.com/images/M/MV5BMTQ5MT...,7.2,8107.0,0.748338
2,3,tt0032138,The Wizard of Oz,G,"Adventure,Family,Fantasy",https://m.media-amazon.com/images/M/MV5BNjUyMT...,8.1,410135.0,0.743003
3,4,tt0031381,Gone with the Wind,Passed,"Drama,Romance,War",https://m.media-amazon.com/images/M/MV5BYjUyZW...,8.2,322258.0,0.873079
4,5,tt0027977,Modern Times,G,"Comedy,Drama,Romance",https://m.media-amazon.com/images/M/MV5BYjJiZj...,8.5,248177.0,0.883686


In [87]:
#cur.execute('''SELECT username FROM users WHERE num_followers > 20''')
cur.execute ('''SELECT tconst from movie_info''')
query_results = cur.fetchall()
tconst_df = pd.DataFrame(query_results)
tconst_df.head

<bound method NDFrame.head of               0
0     tt0011237
1     tt0011237
2     tt0032138
3     tt0031381
4     tt0027977
...         ...
3248  tt2726552
3249  tt4846318
3250  tt5221894
3251  tt4400038
3252  tt4382824

[3253 rows x 1 columns]>

In [154]:
from resmem import ResMem, transformer
model = ResMem(pretrained=True)
model.eval()

for tconst in tconst_df[0][0:10]:
    cur.execute('''SELECT poster_url FROM movie_info WHERE tconst = \'{}\' '''.format(tconst))
    query_results = cur.fetchall()
    np.array(query_results)
    img = read_image_from_s3('images-final-resmem','posters/' + str(tconst) + '.jpg')
    img = img.convert('RGB')
    image_x = transformer(img)
    prediction = model(image_x.view(-1, 3, 227, 227))
    cur.execute('''UPDATE movie_info SET memorability = {} WHERE tconst = \'{}\''''.format(str(prediction.item()),tconst))
    cur.execute('''SELECT memorability FROM movie_info WHERE tconst = \'{}\' '''.format(tconst))
    query_results = cur.fetchall()
    print(query_results)

C:\Users\y8scr\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\y8scr\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


[(0.7483382225036621,), (0.7483382225036621,)]
[(0.7483382225036621,), (0.7483382225036621,)]


KeyboardInterrupt: 

In [110]:
prediction.item()

0.8730794787406921

In [66]:
'''SELECT poster_url FROM movie_info WHERE tconst = {}'''.format(tconst)

'SELECT poster_url FROM movie_info WHERE tconst = i'

In [104]:
'''INSERT INTO movie_info (memorability) VALUES ({})'''.format(prediction.item())

'INSERT INTO movie_info (memorability) VALUES (0.8730794787406921)'

In [107]:
prediction.item()

0.8730794787406921

In [114]:
cur.close()

True

In [160]:
import mysql.connector
conn =  mysql.connector.connect(host=ENDPOINT,
                                user=username,
                                passwd=password, 
                                port=PORT, 
                                database='moviesdb')
cur = conn.cursor()

In [164]:
cur.execute('''SELECT * from movie_info''')
query_results = cur.fetchall()

df_all = pd.DataFrame(query_results)

df_all.iloc[8].isna().sum().sum()

0

In [1]:
cur.close()
conn.close()

NameError: name 'cur' is not defined

In [165]:
df_all.head()

,0,1,2,3,4,5,6,7,8
0,1,tt0011237,"Der Golem, wie er in die Welt kam",Unrated,"Fantasy,Horror",https://m.media-amazon.com/images/M/MV5BMTQ5MT...,7.2,8107,0.741957
1,2,tt0011237,"Der Golem, wie er in die Welt kam",Unrated,"Fantasy,Horror",https://m.media-amazon.com/images/M/MV5BMTQ5MT...,7.2,8107,0.741957
2,3,tt0032138,The Wizard of Oz,G,"Adventure,Family,Fantasy",https://m.media-amazon.com/images/M/MV5BNjUyMT...,8.1,410135,0.742456
3,4,tt0031381,Gone with the Wind,Passed,"Drama,Romance,War",https://m.media-amazon.com/images/M/MV5BYjUyZW...,8.2,322258,0.872611
4,5,tt0027977,Modern Times,G,"Comedy,Drama,Romance",https://m.media-amazon.com/images/M/MV5BYjJiZj...,8.5,248177,0.883703


In [ ]:
DataFrame.columns = new_column_names

In [ ]:
corr = df_all[6].corr(df_all[8])

In [ ]:
corr